In [ ]:
### Load Dataset and PreProcessing for dataset

In [ ]:
from __future__ import division
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
import urllib
import os.path
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import StandardScaler
from collections import namedtuple
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn import feature_extraction
from random import seed, shuffle
from datetime import date
from sklearn.neighbors import KernelDensity
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import distributions
from torch.nn.parameter import Parameter
import torch.utils.data as data_utils
from collections import namedtuple
import functools
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons


### Adult dataset       
def load_adult(scaler=True):
    
    data = pd.read_csv(
        "adult.data",
        names=[
            "Age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
            "occupation", "relationship", "race", "gender", "capital gain", "capital loss",
            "hours per week", "native-country", "income"]
            )
    len_train = len(data.values[:, -1])
    data_test = pd.read_csv(
        "adult.test",
        names=[
            "Age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
            "occupation", "relationship", "race", "gender", "capital gain", "capital loss",
            "hours per week", "native-country", "income"],
        skiprows=1, header=None
    )
    data = pd.concat([data, data_test])
    # Considering the relative low portion of missing data, we discard rows with missing data
    domanda = data["workclass"][4].values[1] # domanda = ?
    data = data[data["workclass"] != domanda]
    data = data[data["occupation"] != domanda]
    data = data[data["native-country"] != domanda]

    # Here we apply discretisation on column marital_status
    data.replace(['Divorced', 'Married-AF-spouse',
                  'Married-civ-spouse', 'Married-spouse-absent',
                  'Never-married', 'Separated', 'Widowed'],
                 ['not married', 'married', 'married', 'married',
                  'not married', 'not married', 'not married'], inplace=True)
    # categorical fields
    category_col = ['workclass', 'race', 'education', 'marital-status', 'occupation',
                    'relationship', 'gender', 'native-country', 'income']
    for col in category_col:
        b, c = np.unique(data[col], return_inverse=True)
        data[col] = c
    
    datamat = data.values
    
    #Care there is a final dot in the class only in test set which creates 4 different classes
    target = np.array([-1.0 if (val == 0 or val==1) else 1.0 for val in np.array(datamat)[:, -1]])
    
    datamat = datamat[:, :-1] # it delete last coloumn
    
    if scaler:
        scaler = StandardScaler()
        scaler.fit(datamat)
        datamat = scaler.transform(datamat)
    
    
    nTrain = len_train
    
    data = namedtuple('_', 'data, target')(datamat[:nTrain, :], target[:nTrain])   # nTrain = 32561
    data_test = namedtuple('_', 'data, target')(datamat[len_train:, :], target[len_train:])  #len_train = 32561
    encoded_data = pd.DataFrame(data.data)
    encoded_data['Target'] = (data.target+1)/2
    to_protect = 1. * (data.data[:,8]!=data.data[:,8][0])
    encoded_data_test = pd.DataFrame(data_test.data)
    encoded_data_test['Target'] = (data_test.target+1)/2
    to_protect_test = 1. * (data_test.data[:,8]!=data_test.data[:,8][0])
    return encoded_data, encoded_data.drop(columns=8), encoded_data_test, encoded_data_test.drop(columns=8) # to_protect,, to_protect_test


### COMPAS dataset
def compute_num_days(s1,s2):
    
    date1 = date(int(s1[0:4]),int(s1[5:7]),int(s1[8:10]))
    date2 = date(int(s2[0:4]),int(s2[5:7]),int(s2[8:10]))
    delta = date2-date1
    return delta.days

def compute_days_series(data):
    num_days = []
    for i in range(data.shape[0]):
        s1 = data["c_jail_in"].iloc[i]
        s2 = data["c_jail_out"].iloc[i]
        if type(s1) is str and type(s2) is str:
            num_days = num_days + [compute_num_days(s1,s2)]
        else:
            num_days = num_days + [None]
    return num_days

def get_unique(data,att):
    column_values = data[att].values.ravel()
    unique_values =  pd.unique(column_values)
    return unique_values

#Encode categorical data to one-hot encoding scheme
def enc_cat_attr(data2,col):
    attr_values = get_unique(data2,col)
    if len(attr_values) > 2:
        for i in attr_values:
            data2[col+'_'+str(i)] = data2[col] == i
        data2 = data2.drop([col],axis=1)
    else:
        enc = preprocessing.LabelEncoder()
        data2[col] = enc.fit_transform(data2[col])
    data_cols = []
    for i in data2.columns:
        if i != 'two_year_recid':
            data_cols = data_cols + [i]
    data_cols = data_cols + ['two_year_recid']
    return data2[data_cols]

def load_compas():
    # Load dataset from file
    data = pd.read_csv(
        "compas-scores-two-years.csv"
            )
    data_size = data.shape
    data_nulls = data.isnull().sum()/data_size[0]
    for i in data_nulls.keys():
        if data_nulls[i] > 0.5:
            data = data.drop(i,axis=1)
    data = data.drop(["decile_score","score_text","v_decile_score","v_score_text"],axis=1)
    y_labels = data["two_year_recid"]
    data = data.drop(["decile_score.1","priors_count.1","is_recid"],axis=1)
    data = data.drop(["id","name","first","last","dob","age","c_case_number","v_screening_date", "v_type_of_assessment","type_of_assessment",
                  "compas_screening_date","days_b_screening_arrest","is_violent_recid","c_charge_desc","screening_date","in_custody",
                  "out_custody","start","end","event","c_offense_date","c_days_from_compas"],axis=1)

    data['num_jail_days'] = pd.Series(compute_days_series(data))
    data = data.drop(["c_jail_in","c_jail_out"],axis=1)
    
    #Filling in missing data for num_jail_days by averaging those with the same label class
    mean_days_recid = int(data.loc[data['two_year_recid'] == 1].num_jail_days.mean())
    mean_days_no_recid = int(data.loc[data['two_year_recid'] == 0].num_jail_days.mean())

    data['num_jail_days'][(data['num_jail_days'].isnull()) & (data['two_year_recid'] == 1)] = mean_days_recid 
    data['num_jail_days'][(data['num_jail_days'].isnull()) & (data['two_year_recid'] == 0)] = mean_days_no_recid
    attr_t = {'sex':'Male', 'race':'Caucasian', 'c_charge_degree':'M'}
    for i in attr_t.keys():
        data[i][data[i] != attr_t[i]] = 0
        data[i][data[i] == attr_t[i]] = 1
    #encoding 'priors_count' where n=0 is class 0, n=1,2 is class 1 and n>2 is class 2
    data['priors_count'][data['priors_count']==0] = 0
    data['priors_count'][(data['priors_count']>0) & (data['priors_count']<=2)] = 1
    data['priors_count'][data['priors_count']>2] = 2

    #encoding 'age_cat' where 'Less than 25' is class 0, '25 - 45' is class 1 and 'Greater than 45' is class 2
    data['age_cat'][data['age_cat']=='Less than 25'] = 0
    data['age_cat'][data['age_cat']=='25 - 45'] = 1
    data['age_cat'][data['age_cat']=='Greater than 45'] = 2

    #encoding 'num_jail_days' where n>=3 is class 0, n=3:13 is class 1 and n>13 is class 2
    data['num_jail_days'][data['num_jail_days']<=3] = 0
    data['num_jail_days'][(data['num_jail_days']>3) & (data['num_jail_days']<=13)] = 1
    data['num_jail_days'][data['num_jail_days']>13] = 2

    #encoding 'juv_fel_counts' where n=0 is class 0 and n>0 in class 1
    data['juv_fel_count'][data['juv_fel_count']==0] = 0
    data['juv_fel_count'][data['juv_fel_count']>0] = 1

    #encoding 'juv_misd_counts' where n=0 is class 0 and n>0 in class 1
    data['juv_misd_count'][data['juv_misd_count']==0] = 0
    data['juv_misd_count'][data['juv_misd_count']>0] = 1

    #encoding 'juv_other_counts' where n=0 is class 0 and n>0 in class 1
    data['juv_other_count'][data['juv_other_count']==0] = 0
    data['juv_other_count'][data['juv_other_count']>0] = 1

    #one-hot encode the categorical attributes with cats > 2
    data = enc_cat_attr(data,'priors_count')
    data = enc_cat_attr(data,'num_jail_days')
    data = enc_cat_attr(data,'age_cat')
    #data = data.drop(['priors_count'],axis=1)

    #replace boolean with float values
    data = data.replace([True,False,1,0],[1.0,0.0,1.0,0.0])
    
    #convert dataset to numpy array
    data_array = np.array(data)
    x = data_array[:,:-1]
    y = data_array[:,-1]
    
    #split data into train and test sets
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)
    
    data = namedtuple('_', 'data, target')(xtrain, ytrain)   # nTrain = 32561
    data_test = namedtuple('_', 'data, target')(xtest, ytest)  #len_train = 32561
    encoded_data = pd.DataFrame(data.data)
    encoded_data['Target'] = data.target
    encoded_data_test = pd.DataFrame(data_test.data)
    encoded_data_test['Target'] = data_test.target
    return encoded_data, encoded_data.drop(columns=1), encoded_data_test, encoded_data_test.drop(columns=1)


### Law dataset
def load_law_school():
    """Law school admission data.
    The Law School Admission Council conducted a survey across 163 law schools in the United
    States. It contains information on 21,790 law students such as their entrance exam scores
    (LSAT), their grade-point average (GPA) collected prior to law school, and their first year
    average grade (FYA).
    Given this data, a school may wish to predict if an applicant will have a high FYA. The school
    would also like to make sure these predictions are not biased by an individual’s race and sex.
    However, the LSAT, GPA, and FYA scores, may be biased due to social factors.
    Example:
        >>> import ethik
        >>> X, y = ethik.datasets.load_law_school()
        >>> X.head(10)
                race     sex  LSAT  UGPA region_first  ZFYA  sander_index
        0      White  Female  39.0   3.1           GL -0.98      0.782738
        1      White  Female  36.0   3.0           GL  0.09      0.735714
        2      White    Male  30.0   3.1           MS -0.35      0.670238
        5   Hispanic    Male  39.0   2.2           NE  0.58      0.697024
        6      White  Female  37.0   3.4           GL -1.26      0.786310
        7      White  Female  30.5   3.6           GL  0.30      0.724107
        8      White    Male  36.0   3.6           GL -0.10      0.792857
        9      White    Male  37.0   2.7           NE -0.12      0.719643
        13     White  Female  37.0   2.6           GL  1.53      0.710119
        14     White    Male  31.0   3.6           GL  0.34      0.730357
        >>> y.head(10)
        0      True
        1      True
        2      True
        5      True
        6      True
        7      True
        8      True
        9     False
        13     True
        14     True
        Name: first_pf, dtype: bool
    References:
        1. https://papers.nips.cc/paper/6995-counterfactual-fairness.pdf
        2. https://github.com/mkusner/counterfactual-fairness
    """
    X = pd.read_csv("law_data.csv", dtype={"race": "category", "region_first": "category"},index_col=0)
    X["sex"] = X["sex"].map(lambda z: 0 if z == 2 else 1)
    X["race"] = X["race"].map(lambda z: 1 if z == "White" else 0)
    X["region_first"] = X["region_first"].map(lambda z: 1 if z == "GL" else 0)
    count_class_0, count_class_1 = X.first_pf.value_counts()

    X_class_0 = X[X['first_pf'] == 0]
    
    X_class_1 = X[X['first_pf'] == 1]
#     print(count_class_0)
    X_class_1_under = X_class_1.sample(1*count_class_1)
    X_under = pd.concat([X_class_1_under, X_class_0], axis=0)
    y = X_under.pop("first_pf").apply(int).astype(bool)
    y = y.replace([True,False],[1.0,0.0])
    
    X = np.array(X_under)
    y = np.array(y)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=42)
    data = namedtuple('_', 'data, target')(xtrain, ytrain)   # nTrain = 32561
    data_test = namedtuple('_', 'data, target')(xtest, ytest)
    encoded_data = pd.DataFrame(data.data)
    encoded_data['Target'] = data.target
    encoded_data_test = pd.DataFrame(data_test.data)
    encoded_data_test['Target'] = data_test.target
    return  encoded_data, encoded_data.drop(columns=0), encoded_data_test, encoded_data_test.drop(columns=0)


### Moon dataset
def load_moon():
    n_train = 10000
    n_test = 5000
    X, Y = make_moons(n_samples=n_train+n_test, noise=0.2, random_state=0)
    Z = np.zeros_like(Y)

    np.random.seed(0)
    for i in range(n_train + n_test):
        if Y[i] == 0:
            if -0.734 < X[i][0] < 0.734:
                Z[i] = np.random.binomial(1, 0.90)
            else:
                Z[i] = np.random.binomial(1, 0.35)
        elif Y[i] == 1:
            if 0.262 < X[i][0] < 1.734:
                Z[i] = np.random.binomial(1, 0.55)
            else:
                Z[i] = np.random.binomial(1, 0.10)
    X = pd.DataFrame(X, columns=['x_1', 'x_2'])
    Y = pd.Series(Y, name='label')
    Z = pd.Series(Z, name='sensitive attribute')

    X_train = X.loc[list(range(10000)), :]
    Y_train = Y.loc[list(range(10000))]
    Z_train = Z.loc[list(range(10000))]

    X_test = X.loc[list(range(10000,15000)), :]
    Y_test = Y.loc[list(range(10000,15000))]
    Z_test = Z.loc[list(range(10000,15000))]

    XZ_train = np.concatenate((X_train, np.reshape(Z_train.values,(-1,1))), axis = 1)
    XZ_test = np.concatenate((X_test, np.reshape(Z_test.values,(-1,1))), axis = 1)
   
    return  X_train, XZ_train, Y_train, X_test, XZ_test, Y_test




In [ ]:
### Enable GPU or CPU

In [ ]:
import torch
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
### Training Process

In [ ]:

sys.path.append(os.path.abspath(os.path.join('')))
from sklearn.neighbors import KernelDensity
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import distributions
from torch.nn.parameter import Parameter
import torch.utils.data as data_utils
from collections import namedtuple
import functools
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

## classifier for CelebA dataset
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet18(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale=False):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet18, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # modified layer4's stride to be 1 instead of 2 bc of smaller 64x64 size
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        self.fc = nn.Linear(2048 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas


### Define the classifier for low dimentional datasets
class Classifier(nn.Module):
    def __init__(self, input_size):
        super(Classifier, self).__init__()
        size = 200
        self.first = nn.Linear(input_size, size)
        self.last = nn.Linear(size, size)    
        self.last2 = nn.Linear(size, 1)
    def forward(self, x):
        out = F.selu(self.first(x))
        out = F.selu(self.last(out))
        out = self.last2(out)
        out = F.sigmoid(out)
        return out

    
### Define the F-divergence estimator    
class T_div(nn.Module):
    def __init__(self):
        super(T_div,self).__init__()
        self.w_params = nn.Sequential (
            nn.Linear(1,5),
            nn.Sigmoid(),
    
            nn.Linear(5,1),
            nn.Sigmoid(),
            )

    def forward(self, x):
        x = self.w_params(x)
        return x

### Training proccess
result_lambda = pd.DataFrame()
for lam in range(10):
    n_seeds = 5
    result_temp = pd.DataFrame()
    for seed in range(n_seeds):
        print('Currently working on - seed: {}'.format(seed))
        # Set a seed for random number generation
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        # Load dataset:
        # Options for datasets are :
        # 1:' COMPAS': load_compas()
        # 2: 'Law School': load_law_school()
        # 3:'Adult': load_adult()
        # 4:'MOON': X_train, XZ_train, Y_train, X_test, XZ_test, Y_test=load_moon()
        # 5:'CelebA': torch.load("filename") # train .pt file / test .pt file 
        encoded_data_xz, encoded_data, encoded_data_test_xz, encoded_data_test=load_compas()
        # Hyper Parameters
        input_size = encoded_data.shape[1]-1 # for Moon: X_train.shape[1]
        num_epochs = 200
        batch_size = 2048
        # Differernt datasets have different learning rate, please see supplementary material
        learning_rate = 6e-4
        cfg_factory=namedtuple('Config', 'model  batch_size num_epochs learning_rate input_size ' )
        config = cfg_factory(Classifier, batch_size, num_epochs, learning_rate, input_size)
        # Load classification model and F-divergence estimator
        model = config.model(config.input_size).to(device)
        T_x = T_div().to(device)
        criterion = nn.BCELoss()
        # Optimizer for classifier and F-divergence estimator
        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=0)#1e-5
        optimizer_T = torch.optim.Adam(T_x.parameters(), lr=config.learning_rate, weight_decay=0)
        # Prepare real world datasets for training
        train_target = torch.tensor(encoded_data['Target'].values.astype(np.long)).long().to(device)
        train_data = torch.tensor(encoded_data.drop('Target', axis = 1).values.astype(np.float32)).to(device)
        train_protect = torch.tensor(encoded_data_xz.drop('Target', axis = 1).values.astype(np.float32)).to(device)
        train_tensor = data_utils.TensorDataset(train_data, train_protect, train_target)
        train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = config.batch_size, shuffle = True)
        # Prepare Moon for training
        # train_target = torch.tensor(Y_train.values.astype(np.long)).long().to(device)
        # train_data = torch.tensor(X_train.values.astype(np.float32)).to(device)
        # train_protect = torch.tensor(XZ_train.astype(np.float32)).to(device)
        # train_tensor = data_utils.TensorDataset(train_data,train_protect, train_target)
        # train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = config.batch_size, shuffle = True)

        # Prepare real world datasets for testing
        target = torch.tensor(encoded_data_test['Target'].values.astype(np.long)).long().to(device)
        data = torch.tensor(encoded_data_test.drop('Target', axis = 1).values.astype(np.float32)).to(device)
        test_data_xz = torch.tensor(encoded_data_test_xz.drop('Target', axis = 1).values.astype(np.float32)).to(device)
        # Prepare Moon for testing
        # target = torch.tensor(Y_test.values.astype(np.long)).long().to(device)
        # data = torch.tensor(X_test.values.astype(np.float32)).to(device)
        # test_data_xz = torch.tensor(XZ_test.astype(np.float32)).to(device)


        # Function to calculate classification accuracy
        def calc_accuracy(outputs,Y): 
            outputs = (outputs >= 0.5).float()
            acc = (outputs == Y).sum().float()/len(Y)
            return acc
        
        # Initialize the value
        epoch_vector = []

        lamda = lam
        di_list = []
        deo_list = []
        test_loss_list = []
        test_accuracy = []
        acc_mean_vector = []
        r2_mean_vector = []
        loss_mean_vector = []
        divergence=0
        loss_regu = 0
        loss = 0
        # Start Training
        for epoch in range(config.num_epochs):
            for i, (x, xz, y) in enumerate(train_loader):
                # Update F-divergence Estimator
                # Notice that for Moon, Adult, COMPAS dataset, F-divergence estimator needs to update j=100 steps;
                # for adult dataset, it needs j=10 steps.
                for j in range(100):
                    T_x.zero_grad()
                    outputs = model(x)
                    
                    
                    # Here we separate the outputs in two groups. 
                    # For different dataset the index for sensitive attributs are different, which can refer from dataset.
                    # The sensitive attribute for COMPAS dataset is 1, i.e., xz[:,1].
                    Tx_output_xz0 = T_x(outputs[xz[:,1]==0])
                    Tx_output_xz1 = T_x(outputs[xz[:,1]!=0])
                    
                    
                    # Make the numbers of the two groups equally
                    min_index = min(len(Tx_output_xz0),len(Tx_output_xz1))
                    Tx_output_xz0_index = torch.randperm(len(Tx_output_xz0))[:min_index]
                    Tx_output_xz1_index = torch.randperm(len(Tx_output_xz1))[:min_index]
                    Tx_output_xz0 = Tx_output_xz0[Tx_output_xz0_index]
                    Tx_output_xz1 = Tx_output_xz1[Tx_output_xz1_index]
                    
                    # NN based F-divergence estimator
                    # For KL divergence
                    mean_P = torch.mean(Tx_output_xz0)
                    mean_Q = torch.mean(torch.exp(Tx_output_xz1-1))
                    ## For Pearson divergence
                    # mean_P = torch.mean(Tx_output_xz0)
                    # mean_Q = torch.mean((0.25*Tx_output_xz1**2+Tx_output_xz1))
                    ## For SH divergence
                    # mean_P = torch.mean(Tx_output_xz0)
                    # mean_Q = torch.mean(Tx_output_xz1/(torch.tensor(1.)-Tx_output_xz1))
                    
                    
                    # DRE F-divergence estimator (same for three divergences)
                    # mean_P = torch.mean(torch.log(Tx_output_xz0+0.0000001))
                    # mean_Q = torch.mean(Tx_output_xz1)-1
                    
                    
                    # Backward process
                    divergence_estimate = mean_P - mean_Q
                    loss_div = -divergence_estimate
                    loss_div.backward()
                    optimizer_T.step()


                # Update Classifier
                model.zero_grad()
                outputs = model(x)
                #Compute the loss of the classification
                loss = criterion(outputs, torch.unsqueeze(y,dim=1).float())
                
                # Here we separate the outputs in two groups. 
                # For different dataset the index for sensitive attributs are different, which can refer from dataset.
                # The sensitive attribute for COMPAS dataset is 1, i.e., xz[:,1].
                outputs_xz0 = outputs[xz[:,1]==0]
                outputs_xz1 = outputs[xz[:,1]!=0]
                Tx_output_xz0 = T_x(outputs_xz0)
                Tx_output_xz1 = T_x(outputs_xz1)
                
                # Make the numbers of the two groups equally
                min_index = min(len(Tx_output_xz0),len(Tx_output_xz1))
                Tx_output_xz0_index = torch.randperm(len(Tx_output_xz0))[:min_index]
                Tx_output_xz1_index = torch.randperm(len(Tx_output_xz1))[:min_index]
                Tx_output_xz0 = Tx_output_xz0[Tx_output_xz0_index]
                Tx_output_xz1 = Tx_output_xz1[Tx_output_xz1_index]

                # Convention method: If we use CON method, we do not use F-divergence estimator. We can directly add regularization term in the loss function
                # You may comment F-divergence estimator training process to accelerate training process
                # y0z0 = torch.mean(outputs_xz0)
                # y0z1 = torch.mean(outputs_xz1)
                # y1z0 = torch.mean(1-outputs_xz0)
                # y1z1 = torch.mean(1-outputs_xz1)
                # # For KL divergence
                # divergence = y0z0*torch.log(y0z0/y0z1)+y1z0*torch.log(y1z0/y1z1)
                # # For Pearson divergence
                # divergence = (y0z0-y0z1)**2/y0z1+(y1z0-y1z1)**2/y1z1
                # # For SH divergence
                # divergence = (pow(y0z0,0.5)-pow(y0z1,0.5))**2+(pow(y1z0,0.5)-pow(y1z1,0.5))**2
                

                # NN based F-divergence estimator
                
                mean_P = torch.mean(Tx_output_xz0)
                mean_Q = torch.mean(torch.exp(Tx_output_xz1-1))
                ## For Pearson divergence
                # mean_P = torch.mean(Tx_output_xz0)
                # mean_Q = torch.mean((0.25*Tx_output_xz1**2+Tx_output_xz1))
                ## For SH divergence
                # mean_P = torch.mean(Tx_output_xz0)
                # mean_Q = torch.mean(Tx_output_xz1/(torch.tensor(1.)-Tx_output_xz1))

                # DRE method
                # For KL divergence
                # mean_P = torch.mean(torch.log(Tx_output_xz0+0.0000001))
                ## For Pearson divergence
                # mean_Q = torch.mean((Tx_output_xz1-1)**2)
                ## For SH divergence        
                # mean_Q = torch.mean((pow(Tx_output_xz1,0.5)-1)**2)
                
                divergence_estimate = abs(mean_P - mean_Q) # abs(mean_P) # abs(mean_Q)
                loss_regu = torch.mean(divergence_estimate)       
                # Regularization Loss w.r.t NN and DRE based estimator is lamda*loss_regu
                # Regularization Loss w.r.t Convention estimator is lamda*divergence
                loss = loss + lamda*loss_regu
                loss.backward()
                optimizer.step()
                loss_mean_vector.append(divergence_estimate)
                acc_mean_vector.append(calc_accuracy(outputs,torch.unsqueeze(y,dim=1)))
                epoch_vector.append(epoch)

        # Testing process
        print(lamda)
        print("Results on test set")
        with torch.no_grad():
            test_outputs = model(data).detach()
            loss = criterion(test_outputs, torch.unsqueeze(target,dim=1).float())
            # Fairness Measurements
            z0y1 = (torch.reshape((test_data_xz[:,1] == 0),(len((test_data_xz[:,1] == 0)),1)) *  (test_outputs >= 0.5)).sum().float() / (test_data_xz[:,1] == 0).sum().float()
            z1y1 = (torch.reshape((test_data_xz[:,1] != 0),(len((test_data_xz[:,1] != 0)),1)) *  (test_outputs >= 0.5)).sum().float() / (test_data_xz[:,1] == 1).sum().float()
            di_1 = z0y1/z1y1
            if di_1 >= 1:
                di_1 = 1/di_1
            deo= abs(z1y1-z0y1)
            result_temp = result_temp.append({"Accuracy":calc_accuracy(test_outputs.cpu(),torch.unsqueeze(target.cpu(),dim=1)),"Diff":deo.cpu(),"Ratio":di_1.cpu()},ignore_index=True)
            print('Accuracy: %.4f, Ratio: %.4f, Diff: %.4f' % (calc_accuracy(test_outputs,torch.unsqueeze(target,dim=1)),di_1,deo))
    result_lambda = result_lambda.append(result_temp.mean(),ignore_index=True)
print(result_lambda)
result_lambda.to_csv("result_compas_dp_kl.csv")

